In [ ]:
import pandas as pd
import numpy as np

In [ ]:
embeddings_file = "<PROJECT_DIR>/output/02_word_embeddings/<DATE>/main.txt"
df_embeddings = pd.read_table(
        embeddings_file,
        skiprows=1,
        sep=" ",
        header=None,
        index_col=0)
df_embeddings.index.names = ['word']
df_embeddings.head()

In [ ]:
evaluation_file_lp = "<PROJECT_DIR>/output/08_propagation_evaluation/<DATE>/main.txt"
evaluation_file_baseline = "<PATH_TO_BASELINE_DF_EVALUATION>"

In [ ]:
chosen_synsets = [15456] 
# chosen_synsets = [9444]
topx = 50

In [ ]:
df_lp = pd.read_csv(
    evaluation_file_lp,
    sep=" ",
    index_col=0)
df_lp = df_lp.sort_values(by="y_conf", ascending=False)
df_lp.head()

In [ ]:
df_baseline = pd.read_csv(
    evaluation_file_baseline,
    sep=" ",
    index_col=0)
df_baseline = df_baseline.sort_values(by="y_conf", ascending=False)
df_baseline.head()

In [ ]:
chosen_words_lp = df_lp.loc[df_lp['y_train'].isin(chosen_synsets)]["y_train"]
chosen_words_baseline = df_baseline.loc[df_baseline['y_train'].isin(chosen_synsets)]["y_train"]

for synset in chosen_synsets:
    chosen_words_lp = pd.concat([chosen_words_lp, df_lp.loc[(df_lp['y_train'] != synset) & (df_lp['y_pred']== synset)]["y_pred"].head(topx)])
    chosen_words_baseline = pd.concat([chosen_words_baseline, df_baseline.loc[(df_baseline['y_train'] != synset) & (df_baseline['y_pred']==synset)]["y_pred"].head(topx)])


print(len(chosen_words_lp))
print(len(chosen_words_baseline))

In [ ]:
chosen_words_lp

In [ ]:
df_lp.loc[df_lp['y_train'].isin(chosen_synsets)]["y_train"].index.tolist()

In [ ]:
words_lp = chosen_words_lp.index.to_frame().reset_index(drop=True)
words_baseline = chosen_words_baseline.index.to_frame().reset_index(drop=True)

words_merged = words_lp.merge(words_baseline, how="outer", indicator=True)
is_training = words_merged["word"].isin(df_lp.loc[df_lp['y_train'].isin(chosen_synsets)]["y_train"].index.tolist())
words_merged.loc[
    is_training, 
    'merge'] = "training"
words_merged.loc[
    ~is_training, 
    'merge'] = words_merged["_merge"]
words_merged = words_merged[["word", "merge"]]
words_merged = words_merged.set_index("word")
print(len(words_merged))
words_merged = words_merged.sort_values(by="merge")
words_merged

In [ ]:
chosen_embeddings = df_embeddings[df_embeddings.index.isin(words_merged.index.tolist())]

In [ ]:
full_list = words_merged.join(chosen_embeddings)
full_list.iloc[:,0:400].head()

In [ ]:
index_without_umlaut = [item.replace(u'ü', 'ue').replace(u'ö', 'oe').replace(u'ä', 'ae').replace(u'ß', 'ss') for item in full_list.index.tolist()]
full_list.index = index_without_umlaut
full_list.iloc[:,1:401].head()

In [ ]:
# export embeddings
full_list.iloc[:,1:401].to_csv("tensorboard_embeddings_thesis_synset_detailed_combined.tsv", sep="\t", header = False, index = False)
# export synsets & words
full_list.iloc[:,0].to_csv("tensorboard_labels_without_umlauts_thesis_synset_detailed_combined.tsv", sep="\t", header = True, index = True)